In [17]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna

In [18]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet' # CAMBIAR EN LA VM

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

In [19]:
data = pl.scan_parquet(dataset_path + dataset_file).filter(pl.col("foto_mes") == 202108).collect()

In [20]:
# convirto a pandas dataframe
data = data.to_pandas()

In [21]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [22]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [23]:
X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

In [8]:
# Cargo el modelo pre-entrenado
model = lgb.Booster(model_file=modelos_path + 'lgb_competencia2_v002.txt')

In [9]:
corte = 10500

In [10]:
y_pred_lgm = model.predict(X_test)

In [11]:
# Agrego la prediccion en el dataset de test
X_test['pred_lgm'] = y_pred_lgm

#  ordeno por probabilidad para marcar a quienes les vamos a enviar el estimulo
idx = np.argsort(y_pred_lgm)[::-1]
X_test.reset_index(drop=True, inplace=True) # reseteo el index para poder ordenar por idx, sino en index tenia por default el numero de fila del dataframe original "data"
X_test = X_test.iloc[idx]

# genero la columna con 1s para los que van a recibir el estimulo, 0 para todo el resto
envios = np.zeros(len(X_test), dtype=int)
envios[:corte] = 1
X_test['Predicted'] = envios

# otra forma seria:
# X_test['Predicted'] = 0
# X_test.loc[:gan_max_idx-1, 'Predicted'] = 1

In [12]:
check = X_test[['numero_de_cliente','Predicted','pred_lgm']]
check

,numero_de_cliente,Predicted,pred_lgm
110078,459159895,1,9.673630e-01
150551,1166078316,1,9.504462e-01
19983,1110574988,1,9.381268e-01
132089,1064730117,1,9.196182e-01
127511,1558057711,1,9.187633e-01
...,...,...,...
159373,674230334,0,7.087513e-07
125930,621548628,0,6.978521e-07
30509,657642857,0,6.548090e-07
60980,666069141,0,5.249640e-07


In [13]:
output = X_test[['numero_de_cliente', 'Predicted']]

file_name = 'results_v02_00.csv'
output_path = base_path + 'exp/competencia_2/' + file_name

output.to_csv(output_path, index=False)

In [14]:
output.shape
output

,numero_de_cliente,Predicted
110078,459159895,1
150551,1166078316,1
19983,1110574988,1
132089,1064730117,1
127511,1558057711,1
...,...,...
159373,674230334,0
125930,621548628,0
30509,657642857,0
60980,666069141,0


### Predicciones para Kaggle

In [ ]:
# Voy a probar 20 puntos de corte para cada semilla (total 100 envios)

version = 'v001' # UPADTE

modelos = [f'lgb_competencia2_{version}_s{semilla}_final.txt' for semilla in semillas]
puntos_corte = np.linspace(9000, 13000, 20).astype(int)

In [25]:
counter = 0

for modelo in modelos:
    
    # cargamos el modelo
    model = lgb.Booster(model_file=modelos_path + modelo)
    
    # predecimos para agosto
    X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()
    y_pred_lgm = model.predict(X_test)
    X_test['pred_lgm'] = y_pred_lgm
    
    # ordeno de mayor probabilidad de baja a menor
    idx = np.argsort(y_pred_lgm)[::-1]
    X_test.reset_index(drop=True, inplace=True)
    X_test = X_test.iloc[idx]
    
    for corte in puntos_corte:
        
        envios = np.zeros(len(X_test), dtype=int)
        envios[:corte] = 1
        X_test['Predicted'] = envios
        
        output = X_test[['numero_de_cliente', 'Predicted']]
        
        counter += 1
        file_name = f'results_{version}_{counter}.csv'
        output_path = base_path + 'exp/competencia_2/' + file_name
        
        output.to_csv(output_path, index=False)
                
        print(f'{output_path} --- modelo: {modelo}, corte: {corte}')

C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_1.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 9000
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_2.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 9210
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_3.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 9421
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_4.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 9631
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_5.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 9842
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_6.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 10052
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_7.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 10263
C:/Eugenio/Maestria/DMEyF/exp/competencia_2/results_v002_8.csv --- modelo: lgb_competencia2_v002_s122219.txt, corte: 10473
C:/Eugenio/Maestria/D